# Analising @WeRateDogs

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
        <li><a href="#gather">Gather</a></li>
        <li><a href="#assess">Assess</a></li>
        <li><a href="#clean">Clean</a></li>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

<a id='wrangling'></a>
## Data Wrangling
<a id='gather'></a>
### Gather

In [1]:
# Importing libraries to be used
import pandas as pd
import numpy as np
import requests
import tweepy
import json

In [2]:
# Importing libraries to gather data from Twitter
import tweepy
import json
from timeit import default_timer as timer

# Variables with access keys to Twitter APIs
consumer_key = 'HNW4WY2zjl76y7rk7uX6Xgl9h'
consumer_secret = 'fKMmVTio5b6DuTL5GE3wnxlzQRzNI7GbX679vWJHp1N2gxqMr6'
access_token = '924852049396846593-bTbqKyGkxblMpc0kg5UojtW9IbH2ezk'
access_secret = 'uNIYZw5wsGf058KKbfXOPkRfBfvzXBtMCIJ5KsPkbn8xc'

# Generating objects from classes needed to connect to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

#### Read data from Udacity (provided by @WeRateDogs)

In [3]:
# Reading archive provided by Udacity with Tweets.
df_tw = pd.read_csv('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv')

In [4]:
df_tw.shape

(2356, 17)

In [5]:
df_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [6]:
sum(df_tw.duplicated())

0

#### Read data from Image Predictions File

In [7]:
df_pred = pd.read_csv('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv', delimiter = '\t')

In [8]:
df_pred.shape

(2075, 12)

In [9]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
df_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [11]:
sum(df_pred.duplicated())

0

#### Read data from Twitter API
Next, the Twitter API is used to query detailed information about the Tweets we have; the IDs of each of the Tweets are used to query the details of each of them. The data obtained comes in JSON format and is stored in the tweet_json.txt file.

In [12]:
# The number of tweets to consult
tweet_ids = df_tw.tweet_id.values
len(tweet_ids)

2356

In [13]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

1: 892420643555336193
Success
2: 892177421306343426
Success
3: 891815181378084864
Success
4: 891689557279858688
Success
5: 891327558926688256
Success
6: 891087950875897856
Success
7: 890971913173991426
Success
8: 890729181411237888
Success
9: 890609185150312448
Success
10: 890240255349198849
Success
11: 890006608113172480
Success
12: 889880896479866881
Success
13: 889665388333682689
Success
14: 889638837579907072
Success
15: 889531135344209921
Success
16: 889278841981685760
Success
17: 888917238123831296
Success
18: 888804989199671297
Success
19: 888554962724278272
Success
20: 888202515573088257
Fail
21: 888078434458587136
Success
22: 887705289381826560
Success
23: 887517139158093824
Success
24: 887473957103951883
Success
25: 887343217045368832
Success
26: 887101392804085760
Success
27: 886983233522544640
Success
28: 886736880519319552
Success
29: 886680336477933568
Success
30: 886366144734445568
Success
31: 886267009285017600
Success
32: 886258384151887873
Success
33: 8860541600590725

KeyboardInterrupt: 

In [14]:
print(end - start)

NameError: name 'end' is not defined

In [59]:
pd.read_json("tweet-json.txt", lines=True)

ValueError: Unexpected character found when decoding 'true'

In [71]:
# Loading text file in JSON format into a Dataframe.
df_tw_details = pd.read_json(open('tweet_json.txt'), lines=True, orient='records')

In [72]:
df_tw_details

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2017-06-17 16:52:05+00:00,876120275196170240,876120275196170240,"Meet Venti, a seemingly caffeinated puppoccino...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 876120269428932608, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
85,2017-06-16 16:11:53+00:00,875747767867523072,875747767867523072,This is Goose. He's a womanizer. Cheeky as h*c...,False,"[0, 125]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 875747755280318464, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
86,2017-06-15 00:13:52+00:00,875144289856114688,875144289856114688,Meet Nugget and Hank. Nugget took Hank's bone....,False,"[0, 133]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 875144175078957056, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN,NaN
87,2017-06-14 21:06:43+00:00,875097192612077568,875097192612077568,You'll get your package when that precious man...,False,"[0, 96]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,0.0,en,NaN,8.744134e+17,8.744134e+17,"{'url': 'https://t.co/PFp4MghzBW', 'expanded':...",{'created_at': 'Mon Jun 12 23:49:34 +0000 2017...


In [16]:
# Passing the content of the text file to a list, to visually review the content in detail,
# including the sub-levels within the JSON file.
data = [json.loads(line) for line in open('tweet_json.txt', 'r')]

In [18]:
data[43]

{'created_at': 'Sun Jul 09 00:00:04 +0000 2017',
 'id': 883838122936631299,
 'id_str': '883838122936631299',
 'full_text': "This is Noah. He can't believe someone made this mess. Got the vacuum out for you though. Offered to help clean pup. 12/10 super good boy https://t.co/V85xujjDDY",
 'truncated': False,
 'display_text_range': [0, 137],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 883838118432034816,
    'id_str': '883838118432034816',
    'indices': [138, 161],
    'media_url': 'http://pbs.twimg.com/media/DEQGFgAXUAAEvfi.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DEQGFgAXUAAEvfi.jpg',
    'url': 'https://t.co/V85xujjDDY',
    'display_url': 'pic.twitter.com/V85xujjDDY',
    'expanded_url': 'https://twitter.com/dog_rates/status/883838122936631299/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 981, 'h': 1200, 'resize': 'fit'},
     'small': {'w'

In [19]:
df_tw_details.shape

(89, 32)

In [20]:
df_tw_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     89 non-null     datetime64[ns, UTC]
 1   id                             89 non-null     int64              
 2   id_str                         89 non-null     int64              
 3   full_text                      89 non-null     object             
 4   truncated                      89 non-null     bool               
 5   display_text_range             89 non-null     object             
 6   entities                       89 non-null     object             
 7   extended_entities              77 non-null     object             
 8   source                         89 non-null     object             
 9   in_reply_to_status_id          3 non-null      float64            
 10  in_reply_to_status_id_str   

Joining the previously joined dataframe with the dataframe containing the data directly from the Twitter API. Considering that the latter dataframe is obtained from the Twitter IDs in the primary file, and also that it contains detailed information that the primary file lacked, it was decided to work with this dataframe with updated data, since in addition to containing missing information in the primary dataset, it does not include information about twenty-five tweets from this dataset. This may seem paradoxical, but the missing tweets are tweets that no longer exist, probably deleted by the users themselves, so leaving them out of the analysis has been considered beneficial to obtain updated results.

Joining dataframes, applying Left Join to add to the primary data, the data corresponding to the breed predictions. However, it is necessary to specify that only the breed with the highest probability of being the one correctly predicted by the neural network from which these data come from has been kept.

In [21]:
# Joining dataframes applying Left Join, leaving on the left the data with the detail of the tweets.
df_merged = pd.merge(df_tw_details, df_pred[['tweet_id', 'jpg_url', 'p1', 'p1_conf', 'p1_dog']], left_on='id', right_on='tweet_id', how='left')

In [22]:
df_merged.shape

(89, 37)

<a id='assess'></a>
### Assess

Once the data has been collected in a single dataframe, the process continues with its evaluation. 

In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 88
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     89 non-null     datetime64[ns, UTC]
 1   id                             89 non-null     int64              
 2   id_str                         89 non-null     int64              
 3   full_text                      89 non-null     object             
 4   truncated                      89 non-null     bool               
 5   display_text_range             89 non-null     object             
 6   entities                       89 non-null     object             
 7   extended_entities              77 non-null     object             
 8   source                         89 non-null     object             
 9   in_reply_to_status_id          3 non-null      float64            
 10  in_reply_to_status_id_str   

Prior to further data evaluation, columns are selected for further work in order to meet the requirements of this project.

In [24]:
df_merged.isna().sum()

created_at                        0
id                                0
id_str                            0
full_text                         0
truncated                         0
display_text_range                0
entities                          0
extended_entities                12
source                            0
in_reply_to_status_id            86
in_reply_to_status_id_str        86
in_reply_to_user_id              86
in_reply_to_user_id_str          86
in_reply_to_screen_name          86
user                              0
geo                              89
coordinates                      89
place                            89
contributors                     89
is_quote_status                   0
retweet_count                     0
favorite_count                    0
favorited                         0
retweeted                         0
possibly_sensitive                6
possibly_sensitive_appealable     6
lang                              0
retweeted_status            

In [25]:
#df_tw[['doggo']].describe()
#pd.value_counts(df_tw['name'])
#df_tw[df_tw['puppo'] != 'None']
#df_tw[df_tw['name'] == 'doggo']

In [32]:
df_merged.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,tweet_id,jpg_url,p1,p1_conf,p1_dog
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,NaN,8.924206e+17,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,orange,0.097049,False
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,NaN,8.921774e+17,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,0.323581,True
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,False,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,NaN,8.918152e+17,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,Chihuahua,0.716012,True
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"[0, 79]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,NaN,8.916896e+17,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,paper_towel,0.170278,False
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,False,"[0, 138]","{'hashtags': [{'text': 'BarkWeek', 'indices': ...","{'media': [{'id': 891327551943041024, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,NaN,NaN,NaN,NaN,NaN,8.913276e+17,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,basset,0.555712,True


In [42]:
pd.json_normalize(data=df_merged, record_path='entities', 
                            meta=df_merged.columns.drop('entities').to_list())

TypeError: string indices must be integers

In [41]:
df_merged.columns.drop('entities').to_list()

['created_at',
 'id',
 'id_str',
 'full_text',
 'truncated',
 'display_text_range',
 'extended_entities',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'possibly_sensitive_appealable',
 'lang',
 'retweeted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'quoted_status',
 'tweet_id',
 'jpg_url',
 'p1',
 'p1_conf',
 'p1_dog']

In [75]:
pd.json_normalize(df_merged['entities'].to_json())


AttributeError: 'str' object has no attribute 'values'

In [76]:
df_merged['entities'].to_json()

'{"0":{"hashtags":[],"symbols":[],"user_mentions":[],"urls":[],"media":[{"id":892420639486877696,"id_str":"892420639486877696","indices":[86,109],"media_url":"http:\\/\\/pbs.twimg.com\\/media\\/DGKD1-bXoAAIAUK.jpg","media_url_https":"https:\\/\\/pbs.twimg.com\\/media\\/DGKD1-bXoAAIAUK.jpg","url":"https:\\/\\/t.co\\/MgUWQ76dJU","display_url":"pic.twitter.com\\/MgUWQ76dJU","expanded_url":"https:\\/\\/twitter.com\\/dog_rates\\/status\\/892420643555336193\\/photo\\/1","type":"photo","sizes":{"thumb":{"w":150,"h":150,"resize":"crop"},"medium":{"w":540,"h":528,"resize":"fit"},"small":{"w":540,"h":528,"resize":"fit"},"large":{"w":540,"h":528,"resize":"fit"}}}]},"1":{"hashtags":[],"symbols":[],"user_mentions":[],"urls":[],"media":[{"id":892177413194625024,"id_str":"892177413194625024","indices":[139,162],"media_url":"http:\\/\\/pbs.twimg.com\\/media\\/DGGmoV4XsAAUL6n.jpg","media_url_https":"https:\\/\\/pbs.twimg.com\\/media\\/DGGmoV4XsAAUL6n.jpg","url":"https:\\/\\/t.co\\/0Xxu71qeIV","display_

In [49]:
pd.json_normalize(, )
#cadena = df_merged.entities.str.split(',', expand = True)
#cadena


KeyError: '0'